<a href="https://colab.research.google.com/github/UdayaPrakashST/BankruptcyModel_SHAP_LLM/blob/main/American_Bankruptcy_Prediction_1_Creating_the_modelling_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/UdayaPrakashST/BankruptcyModel_SHAP_LLM

Cloning into 'BankruptcyModel_SHAP_LLM'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 4.43 MiB | 18.66 MiB/s, done.


In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/BankruptcyModel_SHAP_LLM/american_bankruptcy.csv')